In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import time

import matplotlib.pyplot as plt
import gym
import numpy as np
import pygazebo
from pygazebo.msg.v11.world_control_pb2 import WorldControl
from tqdm.auto import tqdm, trange

from multirotor.helpers import control_allocation_matrix
from multirotor.vehicle import MotorParams, VehicleParams, PropellerParams, SimulationParams
from multirotor.control import PositionController, AttitudeController, PosAttController
from multirotor.control import PosController, AttController, AltController, Controller
from multirotor.simulation import Multirotor, Propeller
from multirotor.visualize import VehicleDrawing
from multirotor.coords import body_to_inertial, direction_cosine_matrix, angular_to_euler_rate
from gym_env import Octorotor
from gazebo_comms import GazeboCommms
from msgs.messages_pb2 import Action, State

### Parameters

In [ ]:
# Tarot T18 params
pp = PropellerParams(
    moment_of_inertia=1.86e-6,
    k_thrust=9.8419e-05,
    k_torque=1.8503e-06
)
vp = VehicleParams(
    propellers=[pp] * 8,
    angles=np.linspace(0, -2*np.pi, num=8, endpoint=False) + 0.375 * np.pi,
    distances=np.ones(8) * 0.635,
    clockwise=[-1,1,-1,1,-1,1,-1,1],
    mass=10.66,
    inertia_matrix=np.asarray([
        [0.2206, 0, 0],
        [0, 0.2206, 0.],
        [0, 0, 0.4238]
    ])
)
sp = SimulationParams(0.001, 9.81)
alloc, alloc_inverse = control_allocation_matrix(vp)

### Python-only

In [ ]:
m = Multirotor(vp, sp)
m.reset()
m.propellers = [Propeller(pp, sp, use_thrust_constant=True) for _ in range(8)]

In [ ]:
# m.state[6:9] = np.asarray([0, np.pi/6, 0])
m.reset()
action = m.allocate_control(m.weight, np.asarray([0, 1e-1, 0]))
for i in range(500): m.step(action)
print(m.orientation * 180 / np.pi)

In [ ]:
# issue with angle calculation formula in physics.py
# allocation works well when controlling fans to get required torque
np.sign(action-action.mean())

In [ ]:
m = Multirotor(vp, sp)
m.reset()
m.propellers = [Propeller(pp, sp, use_thrust_constant=True) for _ in range(8)]
ctrl = Controller(
    PosController(0.1, 0.0, 0., 1., m),
    AttController(np.asarray([10., 10., 0]), 0, 0., 1., m),
    AltController(10,0,0,1,m)
)
targets = np.zeros((8000, 3))
# +y
targets[:2000, 0] = 0.
# targets[:2000, 1] = np.linspace(0, 2., num=2000)
# targets[:2000,2] = np.linspace(0,2, num=2000)
# +x
targets[2000:4000, 0] = 1.
# targets[2000:4000, 1] = 2.
# targets[2000:4000,2] = 2.
# -y
targets[4000:6000, 0] = 1.
# targets[4000:6000, 1] = np.linspace(2., 0., num=2000)
# targets[4000:6000,2] = np.linspace(2,0, num=2000)
# -x
targets[6000:8000, 0] = 0.
# targets[6000:8000, 1] = 0
# targets[6000:8000,2] = 0

positions = []
errs = []
alloc_th = []
alloc_to = []
orientations = []
actions = []
for i in trange(len(targets), leave=False):
    ref = np.asarray([*targets[i], 0.])
    # Get prescribed dynamics for system
    dynamics = ctrl.step(ref)
    thrust, torques = dynamics[0], dynamics[1:]
    errs.append(ctrl.ctrl_p.err)
    alloc_th.append(thrust)
    alloc_to.append(torques)
    # Convert dynamics into motor RPMs
    action = m.allocate_control(thrust, torques)
    actions.append(action)
    m.step(action)
    positions.append(m.position)
    orientations.append(m.orientation)
#     if i==N: break

positions = np.asarray(positions)
errs = np.asarray(errs)
alloc_th = np.asarray(alloc_th)
alloc_to = np.asarray(alloc_to)
orientations = np.asarray(orientations)
actions = np.asarray(actions)

In [ ]:
plt.figure(figsize=(14,9))
plt.subplot(1,2,1)
for p,l, c in zip(positions.T, 'xyz', 'rgb'):
    plt.plot(p, label=l, c=c)
lines = plt.gca().lines
plt.twinx()
for p,l,c in zip(orientations.T, ('roll', 'pitch', 'yaw'), 'cmy'):
    plt.plot(p * 180 / np.pi, label=l, ls=':', c=c)
plt.legend(handles=plt.gca().lines+lines)

plt.subplot(1,2,2)
l = plt.plot(alloc_th, label='Ctrl Thrust')
plt.twinx()
for i, c, a in zip(range(3), 'rgb', 'xyz'):
    plt.plot(alloc_to[:,i], label='Ctrl Torque %s' % a, c=c)
plt.legend(handles=plt.gca().lines + l)

### Control mechanism

In [ ]:
env = Octorotor(gym_fc_obs=False)

In [ ]:
# The relationship between RPM and voltage
vels = []
v_vels = []
for i in range(0, 20):
    env.reset()
    v_vels.append([])
    for _ in range(0, 200):
        vel = env.voltage_to_angular_vel(np.ones(8) * i)[0] * 60 / (2 * np.pi)
        v_vels[-1].append(vel)
    vels.append(vel)

# plt.figure(figsize=(14,5))
# plt.subplot(1,2,1)
# plt.plot(np.arange(0, 20), vels)
# plt.ylabel('RPM after 200ms of constant V signal')
# plt.xlabel('Voltage /V');
# plt.subplot(1,2,2)
# for i, v in enumerate(v_vels):
#     plt.plot(v, label='%d volts' % i, ls=':')
# plt.xlabel('Time /ms')
# plt.ylabel('RPM')
# plt.title('RPM response to voltages')

rpm_voltage_ratio = (vels[-1] - vels[0])[0] / 20.
print(rpm_voltage_ratio)

In [ ]:
# Forces/ torques to rpm
def allocate_control(thrust: float, torques: np.ndarray) -> np.ndarray:
    vec = np.asarray([thrust, *torques])
    return np.sqrt(
        np.clip(alloc_inverse @ vec, a_min=0., a_max=None)
    ) * 60 / (2 * np.pi)
# rpm to voltage
def rpm_to_voltage(rpm):
    return rpm / rpm_voltage_ratio
# voltage in -1 to 1 range (for NNs)
def normalize_voltage(voltage):
    return (voltage - 10) / 20
def rpm_to_rads(rpm):
    return rpm * np.pi / 30

In [ ]:
# RPM to generate thrust to match weight
action_rpm = allocate_control(9.8*10.66, [0,0,0])
print(action_rpm[0])

action_v = rpm_to_voltage(action_rpm)
print(action_v[0])

action_n = normalize_voltage(action_v)
print(action_n[0])

### Message Passing

In [ ]:
gc = GazeboCommms()
gc.manager.publications()

In [ ]:
a = Action()
a.type = a.DYNAMICS
a.values.extend([0., 0., 0., 0., 0., 0.])

# gc.send_TCP(a, '/tarot/action')

In [ ]:
# Reset
a = Action()
a.reset = True
# s = State()
a.state.position.extend([1,0,1])
a.state.velocity.extend([3,0,0])
a.state.orientation.extend([0,0,0])
a.state.angular_rate.extend([0,0,0])
a
# gc.send_TCP(a, '/tarot/action')

In [ ]:
gc.send_UDP(a)

In [ ]:
# For some reason, the first send of TCP message doesnt work
c = WorldControl()
c.reset.all = True
# c.multi_step = 100
gc.send_TCP(c, '/gazebo/test/world_control')

### Simulation

In [ ]:
from gym_env import RemoteOctorotor, LocalOctorotor
from trajectories import Trajectory

In [ ]:
m = Multirotor(vp, sp)
m.propellers = [Propeller(pp, sp, use_thrust_constant=True) for _ in range(8)]
m.reset()
# env = RemoteOctorotor(vehicle=m)
env = RemoteOctorotor(vehicle=m)
traj = Trajectory([[0,0,2], [0,5,2], [5,5,2], [5,0,2], [0,0,2]],
                  env.vehicle, proximity=0.2, resolution=0.25)

In [ ]:
env.reset()

In [ ]:
for _ in range(2000):
    state, *_ = env.step([0., 0., vp.mass * sp.g * 1.001, 0., 0., 0.])

In [ ]:
state

#### Attitude + Position

In [ ]:
# wind = lambda t: [10 * np.sin(t * 2 * np.pi / 4000), 0]
wind = lambda t: [0, 0]

In [ ]:
# m.propellers[2].params.k_thrust = 4.9e-5
# m.propellers[3].params.k_thrust = 4.9e-5

ctrl = Controller(
    PosController(0.5, 0.0, 0.1, 1., m),
    AttController(np.asarray([50., 50., 0.1]), 0, np.asarray([10., 10., 0.1]), 1., m),
    AltController(144,0,24,1,m)
)

targets = []
actions = []
speeds = []
positions = []
velocities = []
orientations = []
errs = []
alloc_errs = []
alloc_th = []
alloc_to = []
thrusts = []

for i, pos in enumerate(traj):
    # Get prescribed dynamics for system
    ref = np.asarray([*pos, 0.])
    dynamics = ctrl.step(ref)
    thrust, torques = dynamics[0], dynamics[1:]
    # Convert dynamics into motor RPMs
    action = m.allocate_control(thrust, torques)
    # Get resultant forces/torques given propeller states
    f, t = m.get_forces_torques(action, state)
    # Add disturbances
    f[:2] = wind(i)
    # Convert RPM to voltage and apply to simulation
    # state, _, done, _ = env.step(normalize_voltage(rpm_to_voltage(action)))
    state, *_ = env.step([*f, *t])
    
    targets.append(pos)
    errs.append(ctrl.ctrl_p.err)
    alloc_th.append(thrust)
    alloc_to.append(torques)
    alloc_errs.append(np.asarray([thrust, *torques]) - m.alloc @ action**2)
    actions.append(action)
    speeds.append([motor.rpm for motor in m.propellers])
    positions.append(state[:3])
    velocities.append(state[3:6])
    orientations.append(state[6:9])

    if np.any(np.abs(m.orientation[:2]) > np.pi/6): break

In [ ]:
targets = np.asarray(targets)
positions = np.asarray(positions)
velocities = np.asarray(velocities)
speeds = np.asarray(speeds)
orientations = np.asarray(orientations) * 180 / np.pi
errs = np.asarray(errs)
alloc_errs = np.asarray(alloc_errs)
alloc_th = np.asarray(alloc_th)
alloc_to = np.asarray(alloc_to)

#### Plotting

In [ ]:
plt.figure(figsize=(14,10))
plt.subplot(3,2,1)

plt.plot(positions[:, 0], label='x', c='r')
plt.plot(targets[:len(positions), 0], c='r', ls=':')
plt.plot(positions[:, 1], label='y', c='g')
plt.plot(targets[:len(positions), 1], c='g', ls=':')
plt.plot(positions[:, 2], label='z', c='b')
lines = plt.gca().lines[::2]
plt.ylabel('Position /m')
plt.twinx()
plt.plot(orientations[:, 0], label='roll', c='c', ls=':')
plt.plot(orientations[:, 1], label='pitch', c='m', ls=':')
plt.plot(orientations[:, 2], label='yaw', c='y', ls=':')
plt.ylabel('Orientation /deg')
plt.legend(handles=plt.gca().lines + lines)

plt.subplot(3,2,2)
actions = np.asarray(actions)
for i in range(actions.shape[1]):
    l, = plt.plot(actions[:,i], label='prop %d' % i)
    plt.plot(speeds[:,i], c=l.get_c())
lines = plt.gca().lines[::2]
plt.legend(handles=lines)
plt.title('Motor speeds /RPM')

plt.subplot(3,2,3)
v_world = np.zeros_like(velocities)
for i, (v, o) in enumerate(zip(velocities, orientations)):
    dcm = direction_cosine_matrix(*o)
    v_world[i] = body_to_inertial(v, dcm)
for i, c, a in zip(range(3), 'rgb', 'xyz'):
#     plt.plot(v_world[:,i], label='Velocity %s' % a, c=c)
    plt.plot(velocities[:,i], label='Velocity %s' % a, c=c)
plt.title('Velocities')

plt.subplot(3,2,4)
plt.title('Controller allocated dynamics')
l = plt.plot(alloc_th, label='Ctrl Thrust')
plt.ylabel('Force /N')
plt.twinx()
for i, c, a in zip(range(3), 'rgb', 'xyz'):
    plt.plot(alloc_to[:,i], label='Ctrl Torque %s' % a, c=c)
plt.ylabel('Torque /Nm')
plt.legend(handles=plt.gca().lines + l)

plt.subplot(3,2,5)
lines = plt.plot(alloc_errs[:, 0], label='Thrust err', c='b')
plt.twinx()
plt.plot(alloc_errs[:, 1], label='Torque x err', ls=':')
plt.plot(alloc_errs[:, 2], label='Torque y err', ls=':')
plt.plot(alloc_errs[:, 3], label='Torque z err', ls=':')
plt.legend(handles = plt.gca().lines + lines)
plt.title('Allocation Errors')

plt.subplot(3,2,6)
plt.plot(targets[:len(positions),0], targets[:len(positions),1], label='Prescribed traj')
plt.plot(positions[:,0], positions[:,1], label='Actual traj')
plt.gca().set_aspect('equal', 'box')
plt.title('XY positions')
plt.legend()

plt.tight_layout()